In [1]:
import pandas as pd
from airtable import airtable

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()
at_api = os.environ["AIRTABLE_API_TOKEN"]
at_base = os.environ["AIRTABLE_DEVREL_BASE_ID"]

columns = ['Name','Alignment','Block Explorer','Announcement Date', 'Mainnet Launch Date','Shutdown Date'
           ,'L1 Standard Bridge Contract', 'L1 NFT Bridge Contract', 'L1 BatchInbox "to address"', 'L1 BatchInbox "from address"','L1 Output Oracle']

In [2]:
at = airtable.Airtable(at_base, at_api)
data = at.get('OP Products')
df = pd.json_normalize(data, record_path='records')
# Rename all columns that start with 'fields.'
df.rename(columns=lambda x: x.replace('fields.', ''), inplace=True)
df = df[columns]

In [3]:
# Filter the DataFrame
filtered_df = df[df['L1 Standard Bridge Contract'].notnull() & df['L1 Standard Bridge Contract'].str.startswith('0x')\
                 & df['L1 BatchInbox "to address"'].notnull() & df['L1 BatchInbox "to address"'].str.startswith('0x')]
filtered_df.reset_index(inplace=True, drop=True)
display(filtered_df)

Name Alignment                    Block Explorer  \
0  OP Mainnet (OVM2)   [Forks]  https://optimistic.etherscan.io/   
1               Aevo    [RAAS]        https://explorer.aevo.xyz/   
2  OP Mainnet (OVM1)   [Forks]  https://optimistic.etherscan.io/   

  Announcement Date Mainnet Launch Date Shutdown Date  \
0               NaN          2021-11-11           NaN   
1        2022-09-28          2023-04-07           NaN   
2               NaN          2021-06-23    2021-11-11   

                  L1 Standard Bridge Contract  \
0  0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1   
1  0x4082C9647c098a6493fb499EaE63b5ce3259c574   
2  0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1   

                       L1 NFT Bridge Contract  \
0  0x5a7749f83b81b301cab5f48eb8516b986daef23d   
1  0x2e7d7B188D663F38c2E4fF9C59328458A2D676F0   
2                                         NaN   

                   L1 BatchInbox "to address"  \
0  0x5E4e65926BA27467555EB562121fac00D24E9dD2   
1  0x253887577420Cb7e7418cD4d50147743c8041b28   
2  0x4BF681894abEc828B212C906082B444Ceb2f6cf6   

                 L1 BatchInbox "from address"  \
0  0x6887246668a3b87F54DeB3b94Ba47a6f63F32985   
1  0x889e21d7BA3d6dD62e75d4980A4Ad1349c61599d   
2  0x6887246668a3b87F54DeB3b94Ba47a6f63F32985   

                             L1 Output Oracle  
0  0xBe5dAb4A2e9cd0F27300dB4aB94BeE3A233AEB19  
1  0x909E51211e959339EFb14b36f5A50955a8ae3770  
2  0xE969C2724d2448F1d1A6189d3e2aA1F37d5998c1

In [4]:
# Post to Dune API
d.write_dune_api_from_pandas(filtered_df, 'op_stack_chain_metadata',\
                             'Metadata about OP Stack Chains (i.e. bridge contracts, submitter contracts, name)')

Response status code: 200
Response content: b'{"success":true}'
